# Installing Magenta
Taken from their [Getting Started](https://colab.research.google.com/notebooks/magenta/hello_magenta/hello_magenta.ipynb#scrollTo=PfRDVhNs3UFx) notebook.

This will take a few minutes to run.

In [ ]:
#@test {"output": "ignore"}
print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print('Importing libraries and defining some helper functions...')
from google.colab import files

import magenta
import note_seq
import tensorflow

print('🎉 Done!')
print(magenta.__version__)
print(tensorflow.__version__)

Installing dependencies...
Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 160837 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Preparing to unpack .../libasound2-dev_1.1.3-5ubuntu0.6_amd64.deb ...
Unpacking libasound2-dev:amd64 (1.1.3-5ubuntu0.6) over (1.1.3-5ubuntu0.5) ...
Preparing to unpack .../libasound2_1.1.3-5ubuntu0.6_amd64.deb ...
Unpacking libasound2:amd64 (1.1.3-5ubuntu0.6) over (1.1.3-5ubuntu0.5) ...
Preparing to unpack .../libasound2-data_1.1.3-5ubuntu0.6_all.deb ...
Unpacking libasound2-data (1.1.3-5ubuntu0.6) over (1.1.3-5ubuntu0.5) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up libasound2-data (1.1.3-5ubuntu0.6) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up libasound2:amd64 (1.1.3-5ubuntu

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


🎉 Done!
2.1.3
2.5.0


# MusicVAE'
This model will serve as the generator for new MIDI as well as the soundtrack variations.

Begin with downloading and extracting the pre-trained model that produces Drums, Melody, and Bass at once.

This will also take a few minutes to run.

In [ ]:
# See the list of checkpoints: https://github.com/magenta/magenta/tree/master/magenta/models/music_vae#pre-trained-checkpoints
import os

if not os.path.isfile("/content/hierdec-trio_16bar.tar"):
    os.system("curl -o hierdec-trio_16bar.tar https://storage.googleapis.com/magentadata/models/music_vae/checkpoints/hierdec-trio_16bar.tar")
    os.system("tar -xvf /content/hierdec-trio_16bar.tar")

if not os.path.isfile("/content/cat-mel_2bar_big.tar"):
    os.system("curl -o cat-mel_2bar_big.tar https://storage.googleapis.com/magentadata/models/music_vae/checkpoints/cat-mel_2bar_big.tar")
    os.system("tar -xvf /content/cat-mel_2bar_big.tar")

In [ ]:
# Initialize the model.
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

music_vae = TrainedModel(
      configs.CONFIG_MAP['hierdec-trio_16bar'], 
      batch_size=4, 
      checkpoint_dir_or_path='/content/hierdec-trio_16bar.ckpt')

mel_2bar_vae = TrainedModel(
    configs.CONFIG_MAP["cat-mel_2bar_big"],
    batch_size=4,
    checkpoint_dir_or_path="/content/cat-mel_2bar_big.ckpt"
)

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, HierarchicalLstmDecoder, and hparams:
{'max_seq_len': 256, 'z_size': 512, 'free_bits': 256, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [1024, 1024], 'enc_rnn_size': [2048, 2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048, 2048]

INFO:tensorflow:
Hierarchical Decoder:
  input length: 256
  level output lengths: [16, 16]

INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]

INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]

INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]



/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Do not call `graph_parents`.
INFO:tensorflow:Restoring parameters from /content/hierdec-trio_16bar.ckpt
INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_deco

# MIDI Generation
Once the models have been loaded, it's a matter of calling the `sample()` function to generate new note sequences. This is the main snipped that makes use of the pre-trained models to generate note sequences. From the note sequences, they are converted into MIDI and played back. I generate 100 samples from the model.

In [ ]:
from google.colab import files

def download_midi(note_sequence, filename):
    note_seq.sequence_proto_to_midi_file(note_sequence, filename)
    files.download(filename)

# this is generating 100 MIDI samples
generated = music_vae.sample(n=100, length=80, temperature=0.8)

for i, sample in enumerate(generated):
    #note_seq.plot_sequence(sample)
    #note_seq.play_sequence(sample, synth=note_seq.fluidsynth)

    download_midi(sample, "eval_trio_sample_temp08_{}.mid".format(i))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

This is just to preview some of the samples. It provides downloads to 25 randomly selected samples generated from above cell


In [ ]:
midi_files = [f for f in glob.glob("/content/eval_trio_sample_temp08_*.mid")]

batch = int(len(midi_files) / 4)

for i in range(batch):
    print(midi_files[i])
    sequence = note_seq.midi_file_to_note_sequence(midi_files[i])
    note_seq.play_sequence(sequence, synth=note_seq.fluidsynth)


# Creating Variations
At this point, I have downloaded the 100 midi samples and curated the most usable and likable ones. I extracted the melodies from two select midi files, upload them, and interpolate a new variation using the MusicVAE model.

In [ ]:
# this is uploading midi files to interpolate between 2 melodies
# upload exactly 2 files please

if not os.path.isfile("Green_Grass_Zone_lvl1 - Copy.mid") or os.path.isfile("Green_Grass_Zone_lvl1.mid"):
    uploaded_files = files.upload()



assert len(uploaded_files) == 2, "Upload exactly 2 files"

"""
upload() returns a dict with a list of file names and their contents:
{
    "file1_path": data,
    "file2_path": data,
    ...
}

so this has to be indexed by file name

"""
assert uploaded_files["Green_Grass_Zone_lvl1 - Copy.mid"]
assert uploaded_files["Green_Grass_Zone_lvl1.mid"]                       # yes, i am interpolating the sample with itself

print(len(uploaded_files["Green_Grass_Zone_lvl1.mid"]))


num_steps = 8

new_sequence = mel_2bar_vae.interpolate(
    uploaded_files["Green_Grass_Zone_lvl1 - Copy.mid"],
    uploaded_files["Green_Grass_Zone_lvl1.mid"],
    num_steps=num_steps,
    length=32
)

interp_sequence = note_seq.sequences_lib.concatenate_sequences(new_sequence)
note_seq.play_sequence(interp_sequence, synth=note_seq.fluidsynth)
note_seq.plot_sequence(interp_sequence)


Saving Green_Grass_Zone_lvl1 - Copy.mid to Green_Grass_Zone_lvl1 - Copy.mid
Saving Green_Grass_Zone_lvl1.mid to Green_Grass_Zone_lvl1.mid
332


'\nnum_steps = 8\n\nnew_sequence = mel_2bar_vae.interpolate(\n    uploaded_files["Green_Grass_Zone_lvl1 - Copy.mid"],\n    uploaded_files["Green_Grass_Zone_lvl1.mid"],\n    num_steps=num_steps,\n    length=32\n)\n\ninterp_sequence = note_seq.sequences_lib.concatenate_sequences(new_sequence)\nnote_seq.play_sequence(interp_sequence, synth=note_seq.fluidsynth)\nnote_seq.plot_sequence(interp_sequence)\n'